In [23]:
import pandas as pd
import glob

#print(glob.glob("/home/jovyan/*"))
allFiles = glob.glob("/home/jovyan/work/data/all_0000000000*")
print(allFiles)
data = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0)
    list_.append(df)
data = pd.concat(list_)


['/home/jovyan/work/data/all_000000000013', '/home/jovyan/work/data/all_000000000014', '/home/jovyan/work/data/all_000000000012', '/home/jovyan/work/data/all_000000000008', '/home/jovyan/work/data/all_000000000001', '/home/jovyan/work/data/all_000000000006', '/home/jovyan/work/data/all_000000000007', '/home/jovyan/work/data/all_000000000000', '/home/jovyan/work/data/all_000000000009', '/home/jovyan/work/data/all_000000000010', '/home/jovyan/work/data/all_000000000011', '/home/jovyan/work/data/all_000000000005', '/home/jovyan/work/data/all_000000000002', '/home/jovyan/work/data/all_000000000003', '/home/jovyan/work/data/all_000000000004']


In [24]:
def filter_train_type(df=[],
                      train_types=[],
                      sum_types = False,
                      sum_columns=['train_count', 'delay', 'total_delay'],
                      aggs={'pressure': 'max',
                            'max_temperature': 'max',
                            'min_temperature': 'min',
                            'mean_temperature': 'mean',
                            'mean_dewpoint': 'mean',
                            'mean_humidity': 'mean',
                            'mean_winddirection': 'mean',
                            'mean_windspeed': 'mean',
                            'max_windspeedms': 'max',
                            'max_windgust': 'max',
                            'max_precipitation1h': 'max',
                            'max_snowdpeth': 'max',
                            'max_n': 'max',
                            'min_vis': 'min',
                            'min_clhb': 'min',
                            'max_precipitation3h': 'max',
                            'max_precipitation6h': 'max'}):
    """
    Filter traintypes from metadata

    train_types     : list
                      list of following options: [0,1,2,3]

                      train_types = {'K': 0,
                                     'L': 1,
                                     'T': 2,
                                     'M': 3}
    sum_types       : bool
                     if True, sum different train types together (default False)

    returns : np array, np array
              labels metadata, labels
    """

    if len(df) == 0:
        return df

    mask = df.loc[:,'train_type'].isin(train_types)
    filt_df = df[(mask)]

    if sum_types:
        d = {}
        for col in sum_columns:
            d[col] = ['sum']
        for col,method in aggs.items():
            d[col] = [method]
        if 'lat' in df:
            d['lat'] = ['max']
        if 'lon' in df:
            d['lon'] = ['max']

        filt_df = filt_df.groupby(['trainstation', 'time'], as_index=False).agg(d)

        filt_df.columns = filt_df.columns.droplevel(1)
        filt_df.drop_duplicates(['trainstation', 'time'], inplace=True)

    return filt_df

data_filt = filter_train_type(data, ['K', 'L'])

In [21]:
print(data_filt)

                            time trainstation train_type  train_count  \
1        2010-01-04 09:00:00 UTC          SPJ          K            1   
2        2012-02-09 12:00:00 UTC          HPK          K            1   
4        2010-01-26 17:00:00 UTC          AHV          K            1   
5        2010-03-16 18:00:00 UTC          TJA          K            1   
6        2010-01-08 09:00:00 UTC           MR          K            1   
9        2010-02-19 21:00:00 UTC           MR          K            1   
10       2010-02-19 16:00:00 UTC          KAJ          K            1   
11       2012-12-23 17:00:00 UTC          LAI          K            1   
14       2016-12-12 17:00:00 UTC          LIS          K            1   
15       2013-03-01 21:00:00 UTC          NRM          K            1   
19       2014-05-19 09:00:00 UTC          IKY          K            1   
20       2012-12-02 21:00:00 UTC          LAI          K            1   
21       2012-01-24 14:00:00 UTC          VAA      

In [25]:
data_filt.sort_values(by=['time', 'trainstation'], inplace=True)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [ ]:
data_filt.to_csv('/home/jovyan/work/data/all_filtered.csv', encoding='utf-8')